 ### 아래 내용은 글로벌 기업 (주)인텔에서 개발한 <br> Intel® AI for Youth Program 내용을 
대한민국 Implementation Partner사인 (주)Brain AI와 Brain AI Lead Coach Network에서 우리나라 초, 중, 고 학생들의 AI 교육을 위해 현지화 한 내용입니다. <br>
(주)Brain AI와 NDA를 체결한 학교에서만 사용 가능하며 NDA를 준수해야합니다. <br>
상업적 사용은 불가하며, 학교내에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: AI 자동 분류기

AI 자동 분류가 우리 생활에 활용 되는 사례를 알아 봅시다. 
1. 수확을 돕는 로봇 Sweper: [링크](https://www.youtube.com/watch?v=5chk9Sory88)
2. 재활용 플라스틱 병 분류: [링크](https://www.youtube.com/watch?v=E30VP9_0M98)

3. TrashBot: [링크](https://www.youtube.com/watch?v=O7DZcaV6MaI)

본격적인 AI 프로젝트 개발에 앞서 간단한 AI 자동 분류기를 만들어 보면서<br>
AI Project Cycle을 좀 더 자세히 알아봅시다.

구글 티처블 머신을 이용하여 AI 모델을 개발하고, 개발이 완료된 AI 모델을 불러와 마이크로비트와 마이크로비트 확장보드, 서보모터로 작동하는 AI 자동 분류기  프로젝트를 개발 해 봅시다.

학생이 만든 시연 영상을 여기 [링크](https://www.youtube.com/watch?v=CLZmixLPicI) 확인 해 봅니다. 

In [1]:
# 필요 라이브러리 불러오기

import cv2
import numpy as np
import time

import serial
import serial.tools.list_ports

import tensorflow.keras
from tensorflow.keras.models import model_from_json

In [2]:
# USB 시리얼 포트 자동 기
ports = serial.tools.list_ports.comports()
com = ''
for port, desc, hwid in sorted(ports):        
    if 'USB' in desc:
        com = port
if com != '':
    print('Micro:bit detected: ', com)   
else:
    print('Please connect your microbit to this PC via USB')   

Micro:bit detected:  COM7


In [3]:
# 시리얼 통신으로 컴퓨터에 연결되어 있는 마이크로비트에 명령 보내기
def SerialSendCommand(cmd):
    cmd = str(cmd) 
    cmd  = cmd + '\n'
    cmd = str.encode(cmd) 
    ser.write(cmd)

In [4]:
#AI 모델 불러오기

model = tensorflow.keras.models.load_model('model/keras_model.h5', compile = False)

In [5]:
#데이터 레이블 불러오기

labels = []
file = open("model/labels.txt", "r")
for x in file:
    labels.append(x.rstrip('\n'))
print('labels = ', labels)
file.close()

labels =  ['0 Brown', '1 Yellow', '2 None']


In [6]:
# 새로운 이미지 데이터 읽어와 분류 예측하기

def ReadPicture(cropped, model):   
    
    input_width = 224
    input_height = 224

    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    normalized_image = (imgRGB.astype(np.float32) / 127.0) - 1
    batch = normalized_image.reshape(1, input_height, input_width, 3)

    prediction = model.predict(batch)
    
    class_id = np.argmax(prediction)
    score = np.max(prediction) 
    
    return class_id, score

In [7]:
# 메인 함수
def Main():
        
    MODE_PAUSE = -1 
    MODE_START = 0
    mode_status = MODE_PAUSE
 
    tic = time.time()
    text = "Press 'p' to start"
    color  = (0, 0, 255)
    cmd = 90
    print("Press [q] to quit") 
  
    video_capture = cv2.VideoCapture(1)
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )

    y1 = int(height - (height * top))
    y2 = int(height - (height * bottom))
    square = (y2-y1)/2
    x1 = int(width/2 - square)
    x2 = int(width/2 + square)
    
    while(True):

        grabbed, frame = video_capture.read()
        cropped = frame[y1:y2, x1:x2]
 
        if mode_status == MODE_START:
        
            if time.time() - tic > 2.5:      
                tic = time.time() 
                
                class_id, score = ReadPicture(cropped, model)   
                text = labels[class_id][1:] + ' ' + str(int(score*100)) + '%' 
                if score >= .6:
                    cmd = labels[class_id][0]
                    a = SerialSendCommand(cmd)
                    print(a)
                    print(text, cmd)
                     
        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        
        cv2.imshow("Frame", frame) 
        
        key = cv2.waitKey(1)
        
        if key == ord("q"):
            break

        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE       
                color = (0, 0, 255)
                text = "Press 'p' to start"
                SerialSendCommand(90)

            elif mode_status == MODE_PAUSE:
                mode_status = MODE_START
                color = (0, 255, 0)
                text = "Press 'p' to pause"
                
    ser.close()
    video_capture.release()
    cv2.destroyAllWindows()
# 설정 값 변경 및 프로그램 실행
ser = serial.Serial(com, 115200, timeout=0,parity=serial.PARITY_NONE, rtscts=0)  

top = .65
bottom = .25

if __name__ == '__main__': 
    Main()

Press [q] to quit
None
 None 99% 2
None
 Yellow 99% 1
None
 None 99% 2
None
 Yellow 89% 1
None
 None 99% 2
None
 Brown 99% 0
None
 Brown 99% 0
None
 None 99% 2


In [8]:
# 설정 값 변경 및 프로그램 실행
ser = serial.Serial(com, 115200, timeout=0,parity=serial.PARITY_NONE, rtscts=0)  

top = .65
bottom = .25

if __name__ == '__main__': 
    Main()

Press [q] to quit
